# Dataframe creation using the Hsiao class

Example notebook showing how to obtain dataframes of generated data (flux and time sample) of supernova 1a by defining some parameters.  

In [1]:
from hsiao import Hsiao
import numpy as np
import pandas as pd
import random as rand
from random import *
import time

### Example of return datafram from Hsiao class for 1 system of 3 images. 

In [2]:
#Hsiao(nb of images, redshift, amplitude, type of plot, magnifications, time delays, time origin, noise level)

# nb of images, time origin are integers
# redshift, amplitude, noise level are floats
# type of plot is a str (choice bewteen : "Flux", "Total_Flux_Without_Noise", "Noise", "Total_Flux_With_Noise")
# magnifications, time delays are arrays of size (nb of images)

H=Hsiao(3, 0.4, 1e-4, "Flux", np.array([1.2, 1.42, 1.52]), np.array([0, 10.34, 24.32]), 55000., 0.05)

The dataframe method from Hsiao class returns 2 different dataframes:  
- the first contains all the given information from the user, e.i all the above parameters
- the second is composed of generated data calculated into Hsiao class. 

To match the first dataframe with the second one, an ID is defined and written in the first column of each dataframe. It is created thanks to the number of images, the amplitude, the redshift and the noise level to make sure there are no duplicates.

In [3]:
f1, f2 = H.dataframe()

In [4]:
f1

,ID,images,time origin,amplitude,time delays,magnifications,redshift,noise level
0,3-0.0001-0.4-0.05,3,55000.0,0.0001,[ 0. 10.34 24.32],[1.2 1.42 1.52],0.4,0.05


In [5]:
f2

,ID,time sample band g,total flux + noise band g,time sample band r,total flux + noise band r,time sample band i,total flux + noise band i
0,3-0.0001-0.4-0.05,54942.48,40.316335,54942.48,-24.899677,54948.88,47.117011
1,3-0.0001-0.4-0.05,54943.28,39.179359,54944.28,-33.462183,54955.08,34.128709
2,3-0.0001-0.4-0.05,54946.08,2.999286,54949.08,-23.544937,54957.28,-13.320069
3,3-0.0001-0.4-0.05,54947.88,-7.685605,54949.88,-55.556874,54962.48,59.671117
4,3-0.0001-0.4-0.05,54950.68,44.584190,54950.68,-8.337510,54966.68,50.012511
...,...,...,...,...,...,...,...
86,3-0.0001-0.4-0.05,55090.28,33.178401,55095.28,28.890817,NaN,NaN
87,3-0.0001-0.4-0.05,55093.08,5.992655,55100.08,56.566083,NaN,NaN
88,3-0.0001-0.4-0.05,55094.88,-1.814373,55100.88,47.287012,NaN,NaN
89,3-0.0001-0.4-0.05,55096.68,35.677426,55104.68,50.621512,NaN,NaN


### Example of dataframe with multiple systems. 

First, the time delays need to be defined such as the first is null and the others are increasing.

In [6]:
def delays(n):
    DT = [0]
    for i in range(1, n):
        DT.append(DT[i-1] + uniform(5, 15))
    return [round(num, 4) for num in DT]

Secondly, to create dataframes containing 20 systems, the parameters are calculated randomly or followinf a unifrom distribution.  
Then, the produced dataframe are concatenate and we obtain 2 dataframes with one composed of all the parameters, and the second the generated data.

In [7]:
start = time.time()

f1 = 0
f2 = 0
f1bis = 0
f2bis = 0

for i in range(5000):
    nb_images = rand.choice([1, 1, 1, 1, 1, 1, 2, 3, 4])  #to maximize non lensed supernova
    redshift = np.around(uniform(1, 2.5), 4)      # do not put more than 2.5
    amplitude = np.around(uniform(1e-5, 1e-1),4)
    mu = [1 + np.around(rand.random(), 4) for i in range(nb_images)]
    time_delays = delays(nb_images)
    #t0 = np.random.randint(55000, 60000)
    t0 = 55000.      #if not, fluxes = csts -> why ? (maybe) because of the def of Hsiao model
    noise_level = np.around(uniform(0, 0.2), 4)
    H = Hsiao(nb_images, redshift, amplitude, "Flux", np.array(mu), np.array(time_delays), t0, noise_level)

    if i == 0:
        f1bis, f2bis = H.dataframe()
    else:
        f1, f2 = H.dataframe()
        f1 = pd.concat([f1bis, f1])
        f2 = pd.concat([f2bis, f2])
        f1bis = f1
        f2bis = f2
end = time.time()
print("The time of execution of above program is :", end-start)

The time of execution of above program is : 130.11095023155212


In [8]:
f1

,ID,images,time origin,amplitude,time delays,magnifications,redshift,noise level
0,1-0.0962-1.4119-0.1595,1,55000.0,0.0962,[0],[1.08],1.4119,0.1595
0,1-0.0137-2.3009-0.0057,1,55000.0,0.0137,[0],[1.41],2.3009,0.0057
0,1-0.0161-1.6411-0.1583,1,55000.0,0.0161,[0],[1.01],1.6411,0.1583
0,2-0.0225-1.7856-0.1731,2,55000.0,0.0225,[ 0. 12.96],[1.77 1.33],1.7856,0.1731
0,4-0.0624-2.3419-0.1966,4,55000.0,0.0624,[ 0. 12.44 27.42 40.71],[1.64 1.48 1.03 1.49],2.3419,0.1966
...,...,...,...,...,...,...,...,...
0,1-0.029-1.8708-0.137,1,55000.0,0.029,[0],[1.94],1.8708,0.137
0,3-0.0201-1.3459-0.0155,3,55000.0,0.0201,[ 0. 7.74 13.09],[1.82 1.54 1.78],1.3459,0.0155
0,1-0.0617-1.6831-0.09,1,55000.0,0.0617,[0],[1.21],1.6831,0.09
0,1-0.0887-1.4188-0.0972,1,55000.0,0.0887,[0],[1.69],1.4188,0.0972


In [9]:
f2

,ID,time sample band g,total flux + noise band g,time sample band r,total flux + noise band r,time sample band i,total flux + noise band i
0,1-0.0962-1.4119-0.1595,54965.8,2286.870275,54966.8,4789.018055,54968.2,60975.710939
1,1-0.0962-1.4119-0.1595,54968.6,8904.996271,54969.6,27607.343911,54973.4,21530.000879
2,1-0.0962-1.4119-0.1595,54971.4,12121.092679,54970.4,-23172.152124,54976.6,38634.842975
3,1-0.0962-1.4119-0.1595,54973.2,-47140.248713,54973.2,504.549232,54979.8,35926.039644
4,1-0.0962-1.4119-0.1595,54974.0,45425.527944,54974.0,5442.310948,54984.0,106884.036868
...,...,...,...,...,...,...,...
86,1-0.07-1.2321-0.0816,55128.6,-10011.038340,55116.6,-1742.088873,NaN,NaN
87,1-0.07-1.2321-0.0816,55129.4,-278.735937,55117.4,1804.681032,NaN,NaN
88,1-0.07-1.2321-0.0816,55132.2,10784.083371,55118.2,3618.485232,NaN,NaN
89,1-0.07-1.2321-0.0816,55136.0,6079.463671,55120.0,-539.731737,NaN,NaN


In [10]:
f1.to_csv("truth.csv", index=False)
f2.to_csv("data.csv", index=False)